## Load Pretrained Model and Tokenizer

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


## Load IMDb Dataset

In [1]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")


c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## Tokenize the Dataset

In [3]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)


## Group Texts into Chunks

In [4]:
chunk_size = tokenizer.mask_token_id

# Drop the last chunk if it’s smaller than chunk_size.
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


# Pad the last chunk until its length equals chunk_size.
def group_texts_with_padding(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute total length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    
    # If the last chunk is smaller than chunk_size, pad it
    for key in result.keys():
        if len(result[key][-1]) < chunk_size:
            padding_length = chunk_size - len(result[key][-1])
            if key == "input_ids":
                # Pad input_ids with the tokenizer's pad token ID
                result[key][-1] += [tokenizer.pad_token_id] * padding_length
            else:
                # Pad other keys (e.g., attention_mask) with 0
                result[key][-1] += [0] * padding_length

    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


In [5]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 76170
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 74448
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 152809
    })
})

In [6]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity'

In [7]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity'

## Combine train and unsupervised Datasets for Training

In [8]:
from datasets import concatenate_datasets

training_dataset = concatenate_datasets([lm_datasets["train"], lm_datasets["unsupervised"]])
evaluation_dataset = lm_datasets["test"]


## Set Up Data Collator

### Individual tokens masking

In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)


In [10]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - yellow from my [MASK] store because [MASK] all the controversy that surrounded it when ivo was first released in [MASK]. [MASK] also heard that at first it [MASK] seized by u. s. customs if [MASK] ever tried departmental enter this country, therefore being a fan of films considered " controversial " i really had [MASK] see [MASK] for [MASK]. < br [MASK] > < br / > the plot is centered around a young swedish drama student named [MASK] who wants to learn everything she can about [MASK] [MASK] in particular'

'>>> [MASK] wants [MASK] focus her [MASK] [MASK] to making some sort of documentary on what the average swede [MASK] about certain political issues such as the vietnam war [MASK] race [MASK] [MASK] the united states. [MASK] [MASK] [MASK] politicians and ordinary denizens [MASK] stockholm about their opinions on politics,uli has sex with her drama magical, classmates, and married men. < br / > < br / > what kills [MASK] about i am curious - yellow i

In [11]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")


'>>> ['[CLS]', '[MASK]', 'rented', 'i', 'am', 'curious', '-', 'yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967', '.', 'i', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', 'u', '.', 's', '.', '##nka', 'if', 'it', 'ever', '[MASK]', 'to', 'enter', 'this', 'country', ',', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '"', 'controversial', '"', 'i', 'really', 'had', 'to', 'see', 'this', 'for', 'myself', '[MASK]', '<', 'georgetown', '/', '>', '<', '[MASK]', '/', '[MASK]', 'the', 'plot', '[MASK]', 'centered', 'around', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', '[MASK]', 'she', 'can', 'about', 'life', '.', '[MASK]', 'particular']'

'>>> ['she', 'wants', 'properties', 'focus', 'her', 'attention', '##s', '[MASK]', 'making', 'some', 'sort', 'of', 'す', 'on', 'what', 'abbott', 'average', 'sw

### Whole word masking

In [12]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [13]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i [MASK] curious - yellow from my [MASK] store because of all the controversy that surrounded it when it [MASK] first released in 1967 [MASK] [MASK] also heard that [MASK] [MASK] it was seized [MASK] u. s. customs [MASK] it ever [MASK] to enter this country, therefore being a fan of [MASK] considered " controversial [MASK] i really [MASK] to see this for [MASK] [MASK] < br / [MASK] < br / > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. [MASK] [MASK]'

'>>> she wants to [MASK] [MASK] [MASK] [MASK] to making some sort of documentary on what the average swede thought [MASK] certain political issues such as the vietnam war and race [MASK] in [MASK] united [MASK]. in between asking politicians [MASK] ordinary denizens of stockholm about their [MASK] on politics, she has sex with her drama teacher [MASK] [MASK], and married [MASK]. < br / > < br / > what kills me about [MASK] am [MASK] - [MASK] is t

In [14]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")


'>>> ['[CLS]', 'i', 'rented', '[MASK]', 'am', 'curious', '-', 'yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', '[MASK]', '[MASK]', 'was', 'first', 'released', 'in', '1967', '.', 'i', 'also', 'heard', 'that', 'at', '[MASK]', 'it', 'was', '[MASK]', 'by', '[MASK]', '.', '[MASK]', '[MASK]', '[MASK]', 'if', 'it', 'ever', 'tried', 'to', 'enter', '[MASK]', '[MASK]', '[MASK]', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '[MASK]', '[MASK]', '"', 'i', 'really', 'had', '[MASK]', 'see', 'this', 'for', 'myself', '[MASK]', '<', 'br', '[MASK]', '[MASK]', '<', 'br', '/', '[MASK]', 'the', 'plot', 'is', 'centered', '[MASK]', '[MASK]', 'young', 'swedish', '[MASK]', 'student', 'named', 'lena', 'who', 'wants', '[MASK]', 'learn', 'everything', '[MASK]', 'can', 'about', 'life', '[MASK]', 'in', 'particular']'

'>>> ['she', 'wants', 'to', 'focus', 'her', 'attention', '##s', 'to', '[MASK]', 'some', 'sort', 'of', 'documentary'

## Define Training Arguments and Initialize Trainer

In [16]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./distilbert-finetuned-imdb-mlm-checkpoint",
    num_train_epochs=10,
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=1000,
    fp16=True,
    logging_steps=100,
    gradient_accumulation_steps=2,
)

# Note: if you're using a whole word masking data collator, you’ll also need to set remove_unused_columns=False to ensure we don’t lose the word_ids column during training. By default, the Trainer will remove any columns that are not part of the model’s forward() method.


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=evaluation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)


c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Train the Model

In [17]:
trainer.train()

  0%|          | 101/35780 [00:13<1:14:13,  8.01it/s]

{'loss': 3.1047, 'grad_norm': 4.89377498626709, 'learning_rate': 1.9600000000000003e-06, 'epoch': 0.03}


  1%|          | 201/35780 [00:25<1:14:02,  8.01it/s]

{'loss': 2.8192, 'grad_norm': 4.3941850662231445, 'learning_rate': 3.96e-06, 'epoch': 0.06}


  1%|          | 301/35780 [00:37<1:13:32,  8.04it/s]

...

                                                       
 59%|█████▊    | 21000/35780 [1:20:36<30:24,  8.10it/s]

{'eval_loss': 2.114932060241699, 'eval_runtime': 52.4896, 'eval_samples_per_second': 1418.337, 'eval_steps_per_second': 44.333, 'epoch': 5.87}


There were missing keys in the checkpoint model loaded: ['vocab_projector.weight'].
 59%|█████▊    | 21000/35780 [1:20:37<56:44,  4.34it/s]

{'train_runtime': 4837.5299, 'train_samples_per_second': 473.339, 'train_steps_per_second': 7.396, 'train_loss': 2.2944495900472006, 'epoch': 5.87}


TrainOutput(global_step=21000, training_loss=2.2944495900472006, metrics={'train_runtime': 4837.5299, 'train_samples_per_second': 473.339, 'train_steps_per_second': 7.396, 'total_flos': 3.583948918699494e+16, 'train_loss': 2.2944495900472006, 'epoch': 5.869200670765791})

## Evaluate the Model

In [18]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


  0%|          | 0/2327 [00:00<?, ?it/s]

100%|██████████| 2327/2327 [00:52<00:00, 43.91it/s]

Perplexity: 8.34


In [19]:
# Saving the Model and Tokenizer

import json
model_output_dir = "distilbert-finetuned-imdb-mlm"

trainer.save_model(model_output_dir)

tokenizer.save_pretrained(model_output_dir)

('distilbert-finetuned-imdb-mlm\\tokenizer_config.json',
 'distilbert-finetuned-imdb-mlm\\special_tokens_map.json',
 'distilbert-finetuned-imdb-mlm\\vocab.txt',
 'distilbert-finetuned-imdb-mlm\\added_tokens.json',
 'distilbert-finetuned-imdb-mlm\\tokenizer.json')

In [22]:
metrics_output_file = model_output_dir + "/log_history.json"
with open(metrics_output_file, "w") as f:
    json.dump(trainer.state.log_history, f)

In [20]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="distilbert-finetuned-imdb-mlm"
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [21]:
text = "This is a great [MASK]."
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great movie.
>>> this is a great film.
>>> this is a great show.
>>> this is a great documentary.
>>> this is a great story.
